In [1]:
from __future__ import division
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
import numpy as np
import operator
from gensim.models import Word2Vec
import sys
sys.path.append('/var/sparkamplify/data_process/')
import db
from article_tagging_min import *
from flashtext import KeywordProcessor
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
#import nltk
import string
from multiprocessing import cpu_count
import time
from math import sqrt, log
import os
db_driver = '/var/sparkamplify/data_process/article_tagging_model/spark-warehouse/mysql-connector-java-8.0.11.jar'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path {} --jars {} pyspark-shell'.format(db_driver, db_driver)
import pickle
from datetime import datetime as dt
import findspark
findspark.init('/opt/spark')
from pyspark import SparkContext, SQLContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import DataFrameReader as dfr
#from spark_sklearn import GridSearchCV

In [2]:
def dictsort(dicty,reverse=False,value=True):
    if value:
        return sorted(dicty.items(), key=operator.itemgetter(1),reverse=reverse)
    else:
        return sorted(dicty.keys(), key=operator.itemgetter(1),reverse=reverse)

In [3]:
def title_proc(title, return_str=True):
    try:
        if return_str:
            return ','.join([stemmer(i) for i in word_tokenize(kwrp.replace_keywords(re.sub('[-/\']+','',title.lower())))
                             if (i not in stop)])
        else:
            return [stemmer(i) for i in word_tokenize(kwrp.replace_keywords(re.sub('[-/\']+','',title.lower()))) if (i not in stop)]
    except AttributeError:
        return ''

In [4]:
def text_keyword_train(train_text,vocabulary =None):
    td=TfidfVectorizer(stop_words=stop_words_list, min_df=3, vocabulary = vocabulary, ngram_range=(1,2))
    tdm=td.fit_transform(train_text)
    ind_vocab={v:k for k,v in td.vocabulary_.items()}
    return td, tdm.tocoo(), ind_vocab

def text_keyword_transform(target_text_list, td_model, fq_model):
    return len(target_text_list), td_model.transform(target_text_list).tocoo(), fq_model.transform(target_text_list).tocoo()

def text_keywords(tdm_coo, fqm_coo, row_len, ind_vocab, topn=5):
    kw_list=[]
    for row in range(row_len):
        if row not in tdm_coo.row:
            print('no text')
            kw_list.append('')
        else:
            fq_data=fqm_coo.data[np.where(fqm_coo.row==row)]
            fq_col=fqm_coo.col[np.where(fqm_coo.row==row)]
            kw_dict={}
            for i in zip(tdm_coo.col[np.where(tdm_coo.row==row)],tdm_coo.data[np.where(tdm_coo.row==row)]):
                fq=fq_data[np.where(fq_col==i[0])]
                if fq>0:
                    wt = 0.5
                else:
                    wt = 0
                    
                if ind_vocab[i[0]].find(' ')>0: #bigram
                    kw_dict[ind_vocab[i[0]]]=(1+i[1])*2.5+wt
                else:
                    kw_dict[ind_vocab[i[0]]]=(1+i[1])*1+wt

            if len(kw_dict)<topn:
                kws=','.join(dictsort(kw_dict, reverse=True, value=False))
            else:
                kws=','.join(dictsort(kw_dict, reverse=True, value=False)[:topn])
            print(kws)
            kw_list.append(kws)
    return kw_list

In [5]:
file_date='{:02d}{:02d}'.format(dt.today().month,dt.today().day)
file_date

'0510'

In [6]:
stemmer = WordNetLemmatizer().lemmatize
stop = stopwords.words('english') + list(string.punctuation)

In [7]:
kwrp=KeywordProcessor(case_sensitive=True)
kwsyn = {'ai':['artificial intelligence'],
         'ar':['augmented reality'],
         'smartglass':['google glass'],
         'biotech':['bioscience','biotech company'],
         'blockchain':['blockchains','blockchain technology'],
         'cryptocurrency':['cryptocurrencies','litecoin'],
         'ethereum':['eth'],
         'medium':['media'],
         'retail':['retailer'],
         'data science':['data scientist'],
         'drone':['drone delivery','delivery drone','drones']}
kwrp.add_keywords_from_dict(kwsyn)
kwrp.get_all_keywords()

{'artificial intelligence': 'ai',
 'augmented reality': 'ar',
 'google glass': 'smartglass',
 'bioscience': 'biotech',
 'biotech company': 'biotech',
 'blockchains': 'blockchain',
 'blockchain technology': 'blockchain',
 'cryptocurrencies': 'cryptocurrency',
 'litecoin': 'cryptocurrency',
 'eth': 'ethereum',
 'media': 'medium',
 'retailer': 'retail',
 'data scientist': 'data science',
 'drone delivery': 'drone',
 'drones': 'drone',
 'delivery drone': 'drone'}

In [8]:
pickle.dump(kwrp, open('../category_reference/keyword_replacer.pickle','wb'))

In [9]:
print("Stop word building ...")
stop_words_list = stopword_building()

#General Processing Building
print("Bigrams building ...")
bigram_list = bigram_proc()  

Stop word building ...
making stopword_list
Bigrams building ...
making bigram_list


In [10]:
def freq_dict(corpus):
    CV=CountVectorizer(min_df=1,ngram_range=(1,2),token_pattern='(?u)[^(,\s)]\w*\s*\w*[^(,\s)]')
        
    voc_freq=CV.fit_transform(corpus).tocoo()
    vocab=CV.vocabulary_
    voc_reverse = {v:k for k,v in vocab.items()}

    total_count=np.unique(voc_freq.col,return_counts=True)
    for voc_ind,count in zip(total_count[0],total_count[1]):
        voc_count[voc_reverse[voc_ind]]=count
    return voc_count

def countmetrix(corpus,stop_words=stop_words_list, min_df=10,vocabulary=None):
    cectorizer=CountVectorizer(stop_words=stop_words, min_df=min_df,vocabulary=vocabulary,ngram_range=(1,2),
                               token_pattern='(?u)[^(,\s)]\w*\s*\w*[^(,\s)]')
    y_coo=cectorizer.fit_transform(corpus).tocoo()
    vocab=cectorizer.vocabulary_
    
    voc_reverse = {v:k for k,v in vocab.items()}
    voc_count = {}
    total_count=np.unique(y_coo.col,return_counts=True)
    for voc_ind,count in zip(total_count[0],total_count[1]):
        voc_count[voc_reverse[voc_ind]]=count
        
    return y_coo, vocab, voc_count

def cat_keyword(y_coo, vocab, voc_count, pos=None, neg=None, how_pos='union', 
                topn=20, min_df=10, reverse=False, rank = False):
    # pos = postive keyword list
    # how_pos ={'union', 'intersection'}
    # neg = negative keyword list
    
    if (not pos):
        print('please give postive kw')
    else:
        if pos: #add bigram process
            ind=0
            for cat in pos:
                cat = stemmer(cat)
                if cat in vocab:
                    if ind==0:
                        articles = y_coo.row[np.where(y_coo.col==vocab[cat])]
                        ind+=1
                    else:
                        if how_pos=='union':
                            articles = y_coo.row[np.where((y_coo.col==vocab[cat])|(np.isin(y_coo.row,articles)))]
                        elif how_pos=='intersection':
                            articles = y_coo.row[np.where((y_coo.col==vocab[cat])&(np.isin(y_coo.row,articles)))]
                        else:
                            print('pls choose how to select from positive ketwords /nhow_pos =\{\'union\', \'intersection\'\}')
                else:
                    print(cat+' is not in dict')
            if ind == 0:
                return 'no valid postive keywords'
                 
        if neg:
            for ncat in neg:
                ncat = stemmer(ncat)
                neg_articles =y_coo.row[np.where(y_coo.col==vocab[ncat])]
                articles = articles[np.where(np.isin(articles,neg_articles,invert=True))]
                
        kw_collect = y_coo.col[np.where(np.isin(y_coo.row,articles))]
                                      
        kw_count=np.unique(kw_collect,return_counts=True)
        voc_reverse = {v:k for k,v in vocab.items()}
        kw_prob={}
        for voc_ind, count in zip(kw_count[0],kw_count[1]):
            if not voc_reverse[voc_ind].isdigit():
                if (voc_count[voc_reverse[voc_ind]]>=min_df) and (count>2) \
                and (log(count/(voc_count[voc_reverse[voc_ind]])+1)*sqrt(count)>0.05) :
                    if reverse:
                        kw_prob[voc_reverse[voc_ind]]=(log(count/len(np.unique(articles))+1)*sqrt(count),count,
                                                   voc_count[voc_reverse[voc_ind]]) 
                        
                    else:
                        kw_prob[voc_reverse[voc_ind]]=(log(count/(voc_count[voc_reverse[voc_ind]])+1)*sqrt(count),count,
                                                       voc_count[voc_reverse[voc_ind]]) 
                                      
        if rank:
            return dictsort(kw_prob,reverse=True)[:topn]
        else:
            return kw_prob

In [11]:
conf = (SparkConf()
  .setAppName("data_import")
  .set("spark.dynamicAllocation.enabled","true")
  .set("spark.shuffle.service.enabled","true")
        .set('spark.executor.memory','32g').set('spark.driver.memory','5g'))

In [12]:
sc = SparkContext(conf=conf)
sc.addPyFile('/var/sparkamplify/data_process/article_tagging_min.py')
sc.addPyFile('/var/sparkamplify/data_process/db.py')
sc.setLogLevel("WARN")
from article_tagging_min import *
sqlContext = SQLContext(sc)

In [13]:
print(sc.getConf().getAll())

[('spark.driver.extraClassPath', '/var/sparkamplify/data_process/article_tagging_model/spark-warehouse/mysql-connector-java-8.0.11.jar'), ('spark.app.id', 'local-1525937312608'), ('spark.shuffle.service.enabled', 'true'), ('spark.jars', 'file:///var/sparkamplify/data_process/article_tagging_model/spark-warehouse/mysql-connector-java-8.0.11.jar'), ('spark.executor.id', 'driver'), ('spark.driver.memory', '5g'), ('spark.driver.port', '43341'), ('spark.executor.memory', '32g'), ('spark.app.name', 'data_import'), ('spark.rdd.compress', 'True'), ('spark.repl.local.jars', 'file:///var/sparkamplify/data_process/article_tagging_model/spark-warehouse/mysql-connector-java-8.0.11.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.deployMode', 'client'), ('spark.dynamicAllocation.enabled', 'true'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.host', 'data-workstation.asia-east1-a.c.spatial-path-187708.internal')]


In [14]:
start=time.time()
df = sqlContext.read.format('jdbc').options(
    url='jdbc:mysql://data-pool.sparkamplify.com:3306/data_pool?autoReconnect=true&useSSL=false',
    dbtable='articles',
    #driver='com.mysql.jdbc.Driver',
    user='harry',
    password='G3LefZtujVWrc8v5').load()
print(time.time()-start)

4.739526748657227


In [15]:
df= df.select('id','title','text','keyword')

In [16]:
df.where(df['keyword'].isNull()).select('id','title','keyword').show(5)

+---+--------------------+-------+
| id|               title|keyword|
+---+--------------------+-------+
|  1|{text} {text} {te...|   null|
|  2|{text} {text} {te...|   null|
|  3|{text} {text} {te...|   null|
|  4|Epson iPrint adds...|   null|
|  5|{text} {text} {te...|   null|
+---+--------------------+-------+
only showing top 5 rows



In [17]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col, concat_ws

In [18]:
func1 = udf(article_lemmatizer_proccess, returnType=StringType())

In [19]:
func2 = udf(title_proc, returnType=StringType())

In [20]:
start=time.time()
df=df.withColumn('p_text',func1('text'))
print(time.time() - start)

0.0640709400177002


In [21]:
start=time.time()
df=df.withColumn('p_title',func1('title'))
print(time.time() - start)

0.03645896911621094


In [22]:
start=time.time()
df=df.withColumn('kw_title',func2('title'))
print(time.time() - start)

0.04123258590698242


In [23]:
df.columns

['id', 'title', 'text', 'keyword', 'p_text', 'p_title', 'kw_title']

In [24]:
def spark_col2list(sdf, col):
    return [i[0] for i in sdf.select(col).collect()]

In [25]:
def spark_col_yield(sdf, col):
    for i in sdf.select(col).collect():
        yield i[0]

In [26]:
spark_col2list(df.where(df.id<10),'id')

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [27]:
df=df.withColumn('textlist', concat_ws(' ', df.p_title, df.p_text))

In [28]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- keyword: string (nullable = true)
 |-- p_text: string (nullable = true)
 |-- p_title: string (nullable = true)
 |-- kw_title: string (nullable = true)
 |-- textlist: string (nullable = false)



In [29]:
from splearn.rdd import ArrayRDD
from splearn.feature_extraction.text import SparkCountVectorizer
from splearn.feature_extraction.text import SparkHashingVectorizer
from splearn.feature_extraction.text import SparkTfidfTransformer
from splearn.pipeline import SparkPipeline

In [30]:
dfr = df.rdd

In [31]:
df.columns

['id', 'title', 'text', 'keyword', 'p_text', 'p_title', 'kw_title', 'textlist']

In [32]:
start=time.time()
pdf = df.select('id','kw_title','textlist').toPandas()
print(time.time()-start)

KeyboardInterrupt: 

In [33]:
X=ArrayRDD(df.select('textlist').rdd)

In [34]:
dist_pipeline = SparkPipeline((
    ('vect', SparkHashingVectorizer(ngram_range=(1,1))),
    ('tfidf', SparkTfidfTransformer())
))

In [35]:
result_dist = dist_pipeline.fit_transform(X)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job 3 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:837)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:835)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:835)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1838)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1751)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1924)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1357)
	at org.apache.spark.SparkContext.stop(SparkContext.sc

In [36]:
voc_test=TfidfVectorizer(stop_words=stop_words_list, min_df=3, ngram_range=(1,1))

In [37]:
voc_test.fit(X)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2, localhost, executor driver): java.sql.SQLException: Java heap space
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:127)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:95)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeInternal(ClientPreparedStatement.java:960)
	at com.mysql.cj.jdbc.ClientPreparedStatement.executeQuery(ClientPreparedStatement.java:1019)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD.compute(JDBCRDD.scala:301)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at or

In [25]:
singram=list(voc_test.vocabulary_.keys())

In [26]:
voc_list=singram+list(bigram_list)

In [27]:
model, matrix, voc_dict =text_keyword_train(textlist,vocabulary=voc_list)

In [28]:
pickle.dump(model, open('../category_reference/keyword_extract_model/keyword_extract_{}.pickle'.format(file_date),'wb'))

In [29]:
pickle.dump(voc_dict, open('../category_reference/keyword_extract_model/keyword_ind_dict_{}.pickle'.format(file_date),'wb'))

In [30]:
fq_dict={v:k for k,v in voc_dict.items()}

In [31]:
fq=CountVectorizer(stop_words=stop_words_list,vocabulary=fq_dict,ngram_range=(1,2))

In [32]:
fqm=fq.fit_transform(textlist).tocoo()

In [33]:
fqtitle = fq.transform(titlelist).tocoo()

In [34]:
fq.vocabulary_ == model.vocabulary_

True

In [35]:
pickle.dump(fq, open('../category_reference/keyword_extract_model/keyword_freq_{}.pickle'.format(file_date),'wb'))

In [36]:
len(textlist)

474645

In [37]:
ar.to_csv('processed_text.csv',index=None)

In [38]:
start=time.time()
ar.loc[ar.title.notnull(),'title']=ar.loc[ar.title.notnull(),'title'].apply(lambda x: title_proc(x))
print(time.time() - start)

111.63013124465942


In [39]:
kwinput=(ar.title.fillna('')+','+ar.kw.fillna('')).apply(lambda x: ','.join(set(x.split(',')))).tolist()

In [40]:
start=time.time()
ca, voc, voc_count=countmetrix(kwinput,stop_words=stop,vocabulary=voc_list)
print(time.time() - start)

12.72156810760498


In [41]:
pickle.dump(ca, open('../category_reference/relative_word_matrix/count_matrix_coo_{}.pickle'.format(file_date),'wb'))
pickle.dump(voc, open('../category_reference/relative_word_matrix/count_matrix_vocab_dict_{}.pickle'.format(file_date),'wb'))
pickle.dump(voc_count, open('../category_reference/relative_word_matrix/count_matrix_freq_dict_{}.pickle'.format(file_date),'wb'))

In [42]:
zz = pickle.load(open('../category_reference/category_kw/top20_sim_words0320_2018.pickle','rb'))
cats = list(zz.keys())
cats+=['other','ar/vr']

In [43]:
cats=cats[2:]

In [44]:
bigram_set=[v for v in voc_list if ' ' in v]

In [56]:
cat_keyword(ca, voc, voc_count, pos=['video game'],neg=[],topn=50,min_df=10,how_pos='union',rank=True)

[('video game', (20.220423211307654, 851, 851)),
 ('coolgames', (1.2158082629295737, 7, 12)),
 ('nukem', (0.837840023674875, 5, 11)),
 ('game', (0.6828829273270927, 448, 13663)),
 ('game console', (0.6473430399601906, 17, 100)),
 ('gamergate', (0.502628856561812, 4, 14)),
 ('super mario', (0.4932485837037557, 18, 146)),
 ('aftra', (0.4755831658727877, 6, 28)),
 ('jumanji', (0.470527414478909, 9, 53)),
 ('guitar hero', (0.44628710262841953, 4, 16)),
 ('violent', (0.39212758286183935, 24, 288)),
 ('video', (0.3876546514927312, 217, 8138)),
 ('hedgehog', (0.38756807783595537, 6, 35)),
 ('mcelroy', (0.32068530015035857, 4, 23)),
 ('steven spielberg', (0.31013722458465603, 14, 162)),
 ('nintendos', (0.2866681439264797, 8, 75)),
 ('mothership', (0.2814910430818505, 3, 17)),
 ('console', (0.2650170468819238, 27, 516)),
 ('atari', (0.2493156657050396, 6, 56)),
 ('gamestop', (0.24596947928506982, 5, 43)),
 ('violence', (0.2440025930250723, 37, 904)),
 ('game developer', (0.22597308541951353, 10

In [46]:
sim_kw={}
for cat in cats:
    
    print(cat)
    sim_kw[cat]={}
    if cat == 'other':
        pass
    
    elif cat == 'ar/vr':
        cat_family=['ar','vr','mixed reality']
        for v in bigram_set:
            if 'ar' in v.split(' '):
                cat_family.append(v)
            if 'vr' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,
                                 neg=[],topn=50,min_df=10,how_pos='union')

    elif cat == 'gaming':
        cat_family=['console','esports','videogame']
        for v in bigram_set:
            if 'game' in v.split(' '):
                cat_family.append(v)
            if 'gaming' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,
                                 neg=['throne','sport','sox game'],topn=50,min_df=10,how_pos='union')
           
    elif cat == 'media':
        cat_family=['medium']
        for v in bigram_set:
            if 'medium' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
           
    elif cat == 'biotech':
        cat_family=[cat]
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]=cat_keyword(ca, voc, voc_count, pos=cat_family+['biopharma','gene','genetics'],neg=[],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'mobility':
        cat_family=[cat]
        for v in bigram_set:
            if 'mobile' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family+['mobile'],neg=[],topn=50,min_df=10,how_pos='union')
    elif cat == 'health':
        cat_family=[cat]
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family) #mental #diet
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family+['depression','diet'],neg=[],topn=50,min_df=10,how_pos='union')
           
    elif cat == 'ai':
        cat_family=[cat,'artificial intelligence']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=['weiwei'],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'robotic':
        cat_family=[cat,'robot','humanoid','robotics']
        for v in bigram_set:
            if 'robot' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'design':
        cat_family=[cat,'designer']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
    
    elif cat == 'advertising':
        cat_family=[cat,'advertisement','ad']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
            if 'ad' in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
    
    elif cat == 'fintech':
        cat_family=[cat,'mobile payment','cryptocurrency','mobile wallet','banking app']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
        
    elif cat == 'ecommerce':
        cat_family=[cat,'online store','online sale','shopping experience','online retailer',
                    'online shopping','online retail']
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')
           
    else:
        cat_family=[cat]
        for v in bigram_set:
            if cat in v.split(' '):
                cat_family.append(v)
        print(cat_family)
        sim_kw[cat]= cat_keyword(ca, voc, voc_count, pos=cat_family,neg=[],topn=50,min_df=10,how_pos='union')

ai
['ai', 'artificial intelligence', 'google ai', 'voice ai', 'ai helping', 'intelligence ai', 'ai assistant', 'ai technology', 'ai machine', 'ai lab', 'ai startup']
blockchain
['blockchain', 'beat blockchain', 'blockchain technology']
health
['health', 'health care', 'digital health', 'health insurance', 'mental health', 'public health', 'health record', 'health data', 'health system', 'health plan', 'health benefit', 'electronic health', 'health service', 'health company', 'reproductive health', 'health issue', 'health tool', 'behavioral health', 'personal health', 'health wellness', 'woman health', 'connected health', 'deepmind health']
biotech
['biotech', 'biotech company']
ecommerce
['ecommerce', 'online store', 'online sale', 'shopping experience', 'online retailer', 'online shopping', 'online retail', 'ecommerce company', 'ecommerce giant', 'ecommerce site', 'ecommerce platform']
enterprise
['enterprise', 'enterprise software', 'enterprise iot', 'packard enterprise', 'enterprise

In [50]:
pickle.dump(sim_kw,open('../category_reference/category_kw/dict_relate_words_{}_2018.pickle'.format(file_date),'wb'))

In [51]:
sim_kw.keys()

dict_keys(['ai', 'blockchain', 'health', 'biotech', 'ecommerce', 'enterprise', 'fintech', 'media', 'advertising', 'gaming', 'robotic', 'drone', 'mobility', 'security', 'design', 'other', 'ar/vr'])

In [52]:
file_date

'0425'

'ad \\c'